'''
Graded Challenge 6

Nama  : Nabila Sulistiowati
Batch : CODA-RMT-003

Program ini dibuat untuk melakukan automatisasi pengolahan (cleaning) data text yang berguna untuk pemodelan model analisa sentimen.

'''

In [5]:
!pip install -q "great-expectations==0.18.19"

In [6]:
import great_expectations as ge                                          # Mengimpor library Great Expectations dengan alias ge, digunakan untuk validasi kualitas data.
from great_expectations.dataset.pandas_dataset import PandasDataset      # Mengimpor PandasDataset, yaitu objek khusus dari Great Expectations yang memungkinkan validasi langsung di atas Pandas DataFrame ;dengan objek ini, kita bisa menggunakan metode validasi langsung di dalam dataframe.
import pandas as pd                                                      # Mengimpor Pandas sebagai pd untuk mengelola data dalam bentuk DataFrame.

# Load Data
df = pd.read_csv("/Users/bee/M3-ETL/P2M3_nabila-sulistiowati_data_raw.csv")
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [7]:
# Info struktur Data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

## Eksplorasi Data

In [8]:
# Hitung jumlah missing values per kolom
df.isnull().sum()  

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

In [9]:
# Cek Duplikasi
duplicates = df.duplicated()
print(df[duplicates])

Empty DataFrame
Columns: [Age, Attrition, BusinessTravel, DailyRate, Department, DistanceFromHome, Education, EducationField, EmployeeCount, EmployeeNumber, EnvironmentSatisfaction, Gender, HourlyRate, JobInvolvement, JobLevel, JobRole, JobSatisfaction, MaritalStatus, MonthlyIncome, MonthlyRate, NumCompaniesWorked, Over18, OverTime, PercentSalaryHike, PerformanceRating, RelationshipSatisfaction, StandardHours, StockOptionLevel, TotalWorkingYears, TrainingTimesLastYear, WorkLifeBalance, YearsAtCompany, YearsInCurrentRole, YearsSinceLastPromotion, YearsWithCurrManager]
Index: []

[0 rows x 35 columns]


## Validasi data menggunakan Great Expectations

In [10]:
# Create a data context

from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir='./')

In [11]:
# Load data
df = PandasDataset(df)

In [12]:
!pip install tabulate

In [13]:
from tabulate import tabulate

print(tabulate(df.head(), headers='keys', tablefmt='psql'))

+----+-------+-------------+-------------------+-------------+------------------------+--------------------+-------------+------------------+-----------------+------------------+---------------------------+----------+--------------+------------------+------------+-----------------------+-------------------+-----------------+-----------------+---------------+----------------------+----------+------------+---------------------+---------------------+----------------------------+-----------------+--------------------+---------------------+-------------------------+-------------------+------------------+----------------------+---------------------------+------------------------+
|    |   Age | Attrition   | BusinessTravel    |   DailyRate | Department             |   DistanceFromHome |   Education | EducationField   |   EmployeeCount |   EmployeeNumber |   EnvironmentSatisfaction | Gender   |   HourlyRate |   JobInvolvement |   JobLevel | JobRole               |   JobSatisfaction | MaritalS

In [14]:
#beri nama data source
datasource_name = "IBM-dataset_csv"  
datasource = context.sources.add_pandas(name=datasource_name)

#beri nama data asset
asset_name = "IBM-dataset"  
data_asset = datasource.add_dataframe_asset(name=asset_name)


In [15]:
# Load csv ke asset
asset = datasource.add_csv_asset(
    name="IBM-dataset_raw", filepath_or_buffer="P2M3_nabila-sulistiowati_data_raw.csv"
)

In [16]:
# Build batch request
batch_request = asset.build_batch_request()

In [17]:
# Membuat suite
expectation_suite_name = 'expectation-IBM-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

{
  "expectation_suite_name": "expectation-IBM-dataset",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.19"
  }
}

In [18]:
# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [19]:
'''
Expectation 1 : to be unique
Untuk memastikan nilai di kolom EmployeeNumber adalah uniik yang mempresentasikan setiap employee
'''
validator.expect_column_values_to_be_unique('EmployeeNumber')


Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 1470,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [20]:
'''
Expectation 2 : to be between min_value and max_value
Memastikan bahwa employee yang bekerja di IBM ada pada rentang usia 18 - 64 tahun (sesuai dengan usia produktif bekerja menurut International Labour Organization (ILO))
'''
validator.expect_column_values_to_be_between(
    column="Age",
    min_value=18,
    max_value=64
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 1470,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [21]:
'''
Expectation 3 : to be in set
Memastikan bahwa kolom "Gender" hanya berisi "Male" dan "Female"
'''
validator.expect_column_values_to_be_in_set(
    column="Gender",
    value_set={"Male", "Female"}
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 1470,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [22]:
'''
Expectation 4 : to be in type list
Memastikan MonthlyIncome dan MonthlyRate bertipe data integer
'''
validator.expect_column_values_to_be_of_type(column="MonthlyIncome", type_="int")

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "int64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [23]:
validator.expect_column_values_to_be_of_type(column="MonthlyRate", type_="int")

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "int64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [31]:
'''
Expectation 5 : expect_column_distinct_values_to_be_in_set
Untuk memastikan bahwa satu-satunya nilai unik (distinct values) dalam kolom “Attrition” adalah “Yes” dan “No”
'''
validator.expect_column_distinct_values_to_be_in_set(
    column="Attrition",
    value_set={"Yes", "No"}
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "No",
      "Yes"
    ],
    "details": {
      "value_counts": [
        {
          "value": "No",
          "count": 1233
        },
        {
          "value": "Yes",
          "count": 237
        }
      ]
    }
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [26]:
'''
Expectation 6 : ExpectColumnValuesToNotBeNull
Memastikan kolom EmployeeNumber tidak Null
'''
validator.expect_column_values_to_not_be_null(column="EmployeeNumber")

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 1470,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [27]:
'''
Expectation 7 : ExpectColumnMaxToBeBetween
Memastikan level edukasi berada pada rentang 1 - 5
'''
validator.expect_column_max_to_be_between(
    column="Education",
    min_value=1,
    max_value=5
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 5
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [28]:
# Save into Expectation Suite
validator.save_expectation_suite(discard_failed_expectations=False)

In [29]:
#checkpoint
checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_IBM-dataset',
    validator = validator,
)